In [ ]:
from bs4 import BeautifulSoup as BS
import unittest
import random
from time import sleep
import pdb
import pandas as pd
import csv
import requests
from tqdm import tqdm
from pandas.io.sql import SQLTable
#import webscrape_pd_scrape_patheos as pscrape
import sys
#from passlib.hash import pbkdf2_sha256
try:
    from configparser import ConfigParser
    from configparser import SafeConfigParser
except ImportError:
    from ConfigParser import ConfigParser  # ver. < 3.0
    from ConfigParser import SafeConfigParser
    
HEADERS = {'user-agent': ('Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_5)'
                          'AppleWebKit/537.36 (KHTML, like Gecko)'
                          'Chrome/73.0.3683.114 Safari/537.36'),
                          'referer': 'https://www.patheos.com/blogs/'}

In [ ]:
def main_full_url_scrape():
    blog_dict = fetch_blogs()
    keep = True
    for i in blog_dict:
        while keep == True:
            keep = False
            #for b in blog_dict['progressive-christian-blogs']:
                #print(b)
            #new_list = blog_find_pages(blog_dict[i], i)
            blog_find_pages(blog_dict['buddhist-blogs'], 'buddhist-blogs')
    test_list = pass_dict_list(blog_dict, 'patheos_blogs_pages.ini')
    for i in test_list:
        print(i)
        
p_full_url_scrape()

In [ ]:
def update_system_status_values(file, section, option, value):
    config = ConfigParser()
    config.read(file)
    cfgfile = open(file, 'w')
    if config.has_section(section) == True:
        config.set(section, option, value)
    else:
        config.add_section(section)
        config.set(section, option, value)
    config.write(cfgfile)
    cfgfile.close()

In [ ]:
def exists_config_file(file, section, option):
    config = ConfigParser()
    config.read(file)
    if config.has_option(section, option) == True:
        page_number = int(config.get(section, option))
    else:
        page_number = 1
    return page_number

In [ ]:
def fetch_blogs():
    url = 'http://www.patheos.com/blogs'
    response = requests.get(url)
    soup = BS(response.content, 'html.parser')

    blog_list = []

    for blog in soup.find_all('div', attrs={"class":"related-content clearfix related-content-sm decorated channel-list"}):
        blog_url1 = blog.find('a')
        blog_url2 = blog_url1['href']
        blog_list.append(blog_url2) 

    blog_lists = []

    for i in blog_list:
        split_url = i.rsplit("/")
        blog_name = split_url[3]
        blog_lists.append(blog_name)

    blog_dict = {i:[] for i in blog_lists}

    blog_cat_prefix = "https://www.patheos.com/"

    i = 0    
    for blog_urls in tqdm(blog_dict, desc='Fetch blog urls for each category'):
        if blog_urls:
            query_url = blog_cat_prefix + blog_urls
            #sleep(random.uniform(1, 3))
            subsub_blog = requests.get(query_url)
            soup2 = BS(subsub_blog.content, 'html.parser')
            for blog in soup2.find_all('div', attrs={"class":"author-info"}):
                for blog_url0 in blog.find_all('div', attrs={"class":"title"}):
                    blog_url1 = blog_url0.find('a')
                    blog_url2 = blog_url1['href']
                    blog_dict[blog_urls].append(blog_url2)
            i = i + 1
        else:
            continue
    return blog_dict

In [ ]:
def test_404(blog, page_number, increment, name):
    pn = 0
    return_str = page_number
    continue_on = True
    if page_number < 1:
        pn = increment
        return_str = 0
    else:
        pn = page_number + increment    
    blog_test = requests.get(blog, timeout=5)
    blog_url = blog_test.url
    while continue_on == True:
        blog_page_url = blog_url + "page/" + str(pn)
        if pn < 20000:
            try:
                url_test = requests.get(blog_page_url, timeout=5) #, allow_redirects=True)
                #sleep(random.uniform(1, 3))
                if url_test.status_code != 404:
                    sys.stdout.write('\r' + blog + " | page(" + str(pn) + ")") # | category total: (" + str(t) + ")")
                    return_str = pn
                    pn = pn + increment
                else:
                    continue_on = False                
            except:
                return_str = -1
                continue_on = False
        else:
            return_str = -2
            continue_on = False
    if increment == 1:
        sys.stdout.write('\r' + blog + " | page(" + str(return_str) + ")") # | category total: (" + str(t) + ")")        
    return return_str                         

In [ ]:
def blog_find_pages(blogname_list, list_name):
    c = 0
    t = 1
    b = 1
    url_list = []
    continue_on = True
    for i in blogname_list:
        if i != "":
            try:
                continue_on = True
                page_prefix = "/page/"
                blog_name = i.rsplit("/")
                try:
                    p = int(exists_config_file('patheos_blogs_pages.ini', list_name, blog_name[4]))
                except:
                    continue
                if not p:
                    p = 0
                try:
                    p1 = test_404(i, p, 500, blog_name[4])
                except:
                    continue

                try:
                    sys.stdout.write('\r' + list_name.upper() + " | category number: " + str(b) + "/" + str(len(blogname_list)))
                except:
                    continue

                if ((p1 != -1) and (p1 != -2)):
                    p2 = test_404(i, p1, 100, blog_name[4])
                    p = p1
                    if p2 != -1:
                        p3 = test_404(i, p2, 50, blog_name[4])
                        p = p2
                        if p3 != -1:
                            p4 = test_404(i, p3, 25, blog_name[4])
                            p = p4

                if ((p != -1) and (p != -2)):
                    while continue_on == True:    
                        sys.stdout.write('\r' + list_name.upper() + " | category number: " + str(b) + "/" + str(len(blogname_list)))
                        p = test_404(i, p, 1, blog_name[4])
                        continue_on = False
                    t = t + p
                    b = b + 1
                    update_system_status_values('patheos_blogs_pages.ini', list_name, blog_name[4], str(p))
                else:
                    update_system_status_values('patheos_blogs_pages.ini', list_name, blog_name[4], str(p))
                    b = b + 1
                    sys.stdout.write('\r' + list_name.upper() + " | category number: " + str(b) + "/" + str(len(blogname_list)))
            except:
                p = -1
                update_system_status_values('patheos_blogs_pages.ini', list_name, blog_name[4], str(p))
                sys.stdout.write('\r' + list_name.upper() + " | category number: " + str(b) + "/" + str(len(blogname_list)))
        else:
            update_system_status_values('patheos_blogs_pages.ini', list_name, blog_name[4], str(p))
            sys.stdout.write('\r' + list_name.upper() + " | category number: " + str(b) + "/" + str(len(blogname_list)))

fail_list = []


In [ ]:
def pass_dict_list(blog_dict, file):
    df_lists_results = pd.DataFrame()
    blog_posts = {i:[] for i in blog_dict}
    
    for i in blog_dict:
        #result_list = fetch_page_posts(i, blog_dict[i], file)
        result_list = fetch_page_posts('buddhist-blogs', blog_dict['buddhist-blogs'], file)
        blog_posts[i].append(result_list)
        
    return blog_posts #dictionary    


def fetch_page_posts(list_name, pages_url_list, file):
    p = 1
    url_list = []
    continue_on = True
    page_prefix = "page/"
    
    #return_dict = {}
    cnumber = 1
    for e in pages_url_list:
        if e:
            blog_name = e.rsplit("/")
            pages = exists_config_file(file, list_name, blog_name[4])
            
            blog_test = requests.get(e, timeout=5)
            blog_url = blog_test.url
            tposts = 0
            pull_page = 1
            while pull_page <= pages: 
                blog_page_url = blog_url + page_prefix + str(pull_page)
                #print(blog_page_url)
                sleep(random.uniform(1, 3))
                url_test = requests.get(blog_page_url)
                if url_test != "<Response [404]>":
                    soup2 = BS(url_test.content, 'html.parser')
                    for blog in soup2.find_all('h2', attrs={"class":"entry-title"}):
                        blog_url1 = blog.find('a')
                        blog_url2 = blog_url1['href']
                        #print(blog_url2)
                        url_list.append(blog_url2)
                        tposts += 1
                        #return_dict[list_name].append(blog_url2)
                        sys.stdout.write('\r' + list_name.upper() + ": " + blog_name[4] + " | blog number: " 
                                             + str(cnumber) + "/" + str(len(pages_url_list)) 
                                             + " | page: " + str(pull_page) 
                                             + " | posts: " + str(tposts))
                    pull_page = pull_page + 1
                    sys.stdout.write('\r' + list_name.upper() + ": " + blog_name[4] + " | blog number: "
                                         + str(cnumber) + "/" + str(len(pages_url_list)) 
                                         + " | page: " + str(pull_page) 
                                         + " | posts: " + str(tposts))
                else:
                    continue
                sys.stdout.write('\r' + list_name.upper() + ": " + blog_name[4] + " | blog number: "
                                     + str(cnumber) + "/" + str(len(pages_url_list)) 
                                     + " | page: " + str(pull_page) 
                                     + " | posts: " + str(tposts))
            sys.stdout.write('\r' + list_name.upper() + ": " + blog_name[4] + " | blog number: "
                                 + str(cnumber) + "/" + str(len(pages_url_list)) 
                                 + " | page: " + str(pull_page) 
                                 + " | posts: " + str(tposts))
        #else:
            #break
            cnumber += 1
            df_temp = pd.DataFrame({'Post URL':url_list})
            #print(df_temp)
            csv_name = blog_name[4] + '.csv'
            df_temp.to_csv(csv_name)

    return url_list
    #return return_dict

#test_list = pass_dict_list(blog_dict, 'patheos_blogs_pages.ini')
    
#posts_list = fetch_page_posts(url_list)

In [ ]:
for i in blog_dict:
    print(i)

In [ ]:
# For in file testing
keep = True
for i in blog_dict:
    #while keep == True:
        #keep = False
        #for b in blog_dict['progressive-christian-blogs']:
            #print(b)
        new_list = blog_find_pages(blog_dict[i], i)
        #new_list = blog_find_pages(blog_dict['progressive-christian-blogs'], 'progressive-christian-blogs')

In [ ]:
for i in 
#    print(i)

In [ ]:
# For in file testing
n = 0

for i in blog_dict:
    print(i, len(blog_dict[i]))
    n = n + len(blog_dict[i])

print("Total: " + str(n))

print(blog_dict['buddhist-blogs'])

In [ ]:
# For in file testing
n = 0

for f in blog_dict['progressive-christian-blogs']:
    print(f)
    n = n + len(blog_dict['progressive-christian-blogs'])

print("Total: " + str(n))

In [ ]:
response_list = []

teste = requests.get('https://www.patheos.com/blogs/jesuscrees')
print(teste.status_code)
print(teste.history)
for i in teste.history:
    response_list.append(i.status_code)
    rget = i.status_code

print(len(response_list))    
print(rget)
print(teste.url)

test1 = teste.url

test2 = requests.get(test1)
print(test2.status_code)

test = requests.get

In [ ]:
test = requests.get('https://www.patheos.com/blogs/comingoutchristian')
print(test.status_code)
print(test.history)
for i in test.history:
    rget = i.status_code
print(rget)
print(test.url)



        